![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

# Cargar datos
df_rental = pd.read_csv("rental_info.csv")

# Calcular la duración del alquiler
df_rental["rental_length"] = pd.to_datetime(df_rental["return_date"]) - pd.to_datetime(df_rental["rental_date"])
df_rental["rental_length_days"] = df_rental["rental_length"].dt.days

# Crear variables dummy para las características especiales
df_rental["deleted_scenes"] = np.where(df_rental["special_features"].str.contains("Deleted Scenes"), 1, 0)
df_rental["behind_the_scenes"] = np.where(df_rental["special_features"].str.contains("Behind the Scenes"), 1, 0)

# Definir las columnas a eliminar
columns_to_remove = ["special_features", "rental_length", "rental_length_days", "rental_date", "return_date"]

# Configurar variables de entrada y objetivo
X = df_rental.drop(columns_to_remove, axis=1)
y = df_rental["rental_length_days"]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

# Configurar y ajustar el modelo de Lasso
lasso_model = Lasso(alpha=0.3, random_state=9)
lasso_model.fit(X_train, y_train)
lasso_coefs = lasso_model.coef_

# Seleccionar características con coeficientes positivos de Lasso
X_train_lasso, X_test_lasso = X_train.iloc[:, lasso_coefs > 0], X_test.iloc[:, lasso_coefs > 0]

# Ajustar una Regresión Lineal en las características seleccionadas
ols_model = LinearRegression().fit(X_train_lasso, y_train)
y_pred_ols = ols_model.predict(X_test_lasso)
mse_ols = mean_squared_error(y_test, y_pred_ols)

# Configurar el espacio de hiperparámetros para el modelo de Random Forest
param_grid = {
    'n_estimators': np.arange(1, 101),
    'max_depth': np.arange(1, 11)
}

# Configurar el modelo de Random Forest y buscar los mejores hiperparámetros
rf_model = RandomForestRegressor(random_state=9)
random_search = RandomizedSearchCV(rf_model, param_distributions=param_grid, cv=5, random_state=9)
random_search.fit(X_train, y_train)

# Extraer los mejores hiperparámetros y entrenar Random Forest con ellos
best_params = random_search.best_params_
rf_best = RandomForestRegressor(n_estimators=best_params["n_estimators"], max_depth=best_params["max_depth"], random_state=9)
rf_best.fit(X_train, y_train)
rf_predictions = rf_best.predict(X_test)
mse_rf = mean_squared_error(y_test, rf_predictions)

# Seleccionar el mejor modelo basado en el MSE
best_model, best_mse = (rf_best, mse_rf) if mse_rf < mse_ols else (ols_model, mse_ols)
